In [13]:
parsed_file = "test_file"
word_occurence_map = {}

with open(parsed_file) as inp:
    prev_line, prev_count = '', 0
    for line in inp:
        curr_line = "\t".join(line.strip().split("\t")[:-1])
        if curr_line == prev_line or (not prev_line):
            prev_count += int(line.strip().split("\t")[-1])
        else:
            print (curr_line)
            x, y, path = prev_line.split('\t')
            key = str(x) + '_' + str(y)
            current = path + ":" + str(prev_count)
            if key in word_occurence_map:
                pastkeys = word_occurence_map[key]
                current =  pastkeys + ',' + current
            word_occurence_map[key] = current
            prev_count = 1
        prev_line = curr_line
    x, y, path = prev_line.split('\t')
    key = str(x) + '_' + str(y)
    current = path + ":" + str(prev_count)
    if key in word_occurence_map:
        pastkeys = word_occurence_map[key]
        current =  pastkeys + ',' + current
    word_occurence_map[key] = current

19193456	23452389	9668810
663726	9048846	4107215
6178566	6097931	8733868
8184873	13389052	6536081
5933740	15640897	8593453
1837912	2946514	9443662
16864707	22008180	11071817
13583494	15294065	657327
18679314	8042043	11954734
18679314	8042040	11954736
18679314	8042043	11954739


In [ ]:
from difflib import SequenceMatcher
return max([(word, SequenceMatcher(lambda x: x == " ", word_to_compare, word).ratio()) for word in words], lambda l:float(l[1]))

